In [1]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.model_selection import GridSearchCV
# from sklearn.feature_selection import SelectKBest, f_regression

# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score
# from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# from sklearn.preprocessing import StandardScaler

# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import f_regression
# from sklearn.decomposition import PCA

from dask.distributed import Client, LocalCluster
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split

# import distributed.joblib
# from dask_ml import joblib

import joblib
from joblib import parallel_backend

import timeit

from dask_ml.model_selection import GridSearchCV
from dask_ml.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor


from dask_ml.wrappers import Incremental


In [2]:
from dask_ml.preprocessing import StandardScaler

from dask.diagnostics import ResourceProfiler, Profiler, ProgressBar
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import PassiveAggressiveRegressor

In [3]:
cluster = LocalCluster(threads_per_worker = 1, n_workers = 2)

client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://127.0.0.1:50232 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 8.59 GB


In [5]:
df = pd.read_csv('large_cleaned_data.csv').drop('Unnamed: 0', 1)

In [6]:
X, y = df.drop(['Date', 'Close'], 1), df[['Close']]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)


In [8]:
import dask.array as da

X_train_da = da.from_array(X_train.values, chunks = (1000000, 4))
y_train_da = da.from_array(y_train.values, chunks = (1000000, 1))

X_test_da = da.from_array(X_test.values, chunks = (1000000, 4))
y_test_da = da.from_array(y_test.values, chunks = (1000000, 1))

In [9]:
import dask

X_train_da, y_train_da = dask.persist(X_train_da, y_train_da)
X_test_da, y_test_da = dask.persist(X_test_da, y_test_da)

In [10]:
est = PassiveAggressiveRegressor()
parameters = {'C': [.5,1,2]}

In [11]:
start = timeit.default_timer()
search = GridSearchCV(est, parameters, cv = 3).fit(X_train_da, y_train_da.ravel())
end = timeit.default_timer()

print(end - start)

149.40438881


In [26]:
inc = Incremental(PassiveAggressiveRegressor(C=search.best_estimator_.get_params()['C']))

start = timeit.default_timer()
inc.fit(X_train_da, y_train_da.ravel())
end = timeit.default_timer()

print(end - start)

2.228935674000013


In [27]:
inc.score(X_test_da, y_test_da)

0.9996896951494312